In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from netCDF4 import Dataset, num2date, date2num
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pandas as pd
import pickle

In [3]:
dataset = Dataset("precip.mon.total.v2018.nc")

Source: https://www.esrl.noaa.gov/psd/data/gridded/data.gpcc.html

In [4]:
dataset.variables.keys()

odict_keys(['lat', 'lon', 'time', 'precip'])

In [5]:
dataset.variables['precip']

<class 'netCDF4._netCDF4.Variable'>
float32 precip(time, lat, lon)
    missing_value: -9.96921e+36
    units: mm
    var_desc: Precipitation
    level_desc: Surface
    parent_stat: Observations
    long_name: GPCC Monthly total of precipitation
    valid_range: [   0. 8000.]
    statistic: Total
    level: Surface
    actual_range: [   0.   4552.43]
    dataset: GPCC Precipitation 0.5degree V2018 Full Reanalysis
unlimited dimensions: time
current shape = (1512, 360, 720)
filling on, default _FillValue of 9.969209968386869e+36 used

In [6]:
times = dataset.variables['time']

dates = num2date(times[:], units=times.units)

In [7]:
lat = dataset.variables['lat'][:]
lon = dataset.variables['lon'][:]

In [8]:
precip = dataset.variables['precip'][:]

In [9]:
precip.shape

(1512, 360, 720)

In [10]:
# filtering coordinates of Alberta

idx_lat = np.where((lat >= 48.99) & (lat <= 60))

idx_lon = np.where((lon >= 360-120) & (lon <= 360-110)) # longitudes to interval [0.25, 356.75]

In [11]:
temp = precip[:,:,idx_lon][:,:,0,:]
temp = temp[:,idx_lat,:][:,0,:,:]
print(temp.shape)

(1512, 22, 20)


In [12]:
df1 = pd.DataFrame(columns=['date', 'latitude', 'longitude', 'precip'])

In [143]:
## not needed if the array is masked
# p = temp
# idx_x, idx_y, idx_z = np.where(p < -10^(-20))
# p[idx_x, idx_y, idx_z] = np.nan

In [13]:
i = 0
for d in range(0, len(dates)):
    for x in range(0, 20):
        for y in range(0, 20):
            df2 = pd.DataFrame({'date': dates[d], "latitude": lat[idx_lat][x], 'longitude': lon[idx_lon][y], 'precip': temp[d, x, y]}, index=[i])
            df1 = df1.append([df2])
            i += 1

In [14]:
df1.head()

,date,latitude,longitude,precip
0,1891-01-01,59.75,240.25,21.459999
1,1891-01-01,59.75,240.75,21.139999
2,1891-01-01,59.75,241.25,21.309999
3,1891-01-01,59.75,241.75,21.900000
4,1891-01-01,59.75,242.25,21.900000


In [15]:
len(df1)

604800

In [19]:
outfile = open('precip_ab.pkl','wb')

In [20]:
pickle.dump(df1, outfile)
outfile.close()